In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("./tsdata/modeldata.csv", index_col='Date', parse_dates=True)
df.index.freq = "MS"


scaler = MinMaxScaler()
df[['returns', 'msci']] = scaler.fit_transform(df[['returns', 'msci']])

df['msci_lag1'] = df['msci'].shift(1)
df['sentiment_lag1'] = df['sentiment'].shift(1)
df['msci_lag2'] = df['msci'].shift(2)
df['sentiment_lag2'] = df['sentiment'].shift(2)
df['msci_lag3'] = df['msci'].shift(3)
df['sentiment_lag3'] = df['sentiment'].shift(3)

df.dropna(inplace=True)

train = df[:'2020-01-01']
test = df['2020-01-02':]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


: 

: 

In [83]:
y_train_base = train['returns']
#X = train.drop("returns", axis=1)
#X = train["returns", "msci"]

X_train_base = train[["returns"]]

X_train_msci = train[["returns", "msci", "msci_lag1", "msci_lag2", "msci_lag3"]]

X_train_sent = train[["returns", "sentiment", "sentiment_lag1", "sentiment_lag2", "sentiment_lag3"]]

X_train_both = train[["returns", "sentiment", "sentiment_lag1", "sentiment_lag2", "sentiment_lag3", "msci", "msci_lag1", "msci_lag2", "msci_lag3"]]


y_test = test['returns']
#X = train.drop("returns", axis=1)
#X = train["returns", "msci"]

X_test_base = test[["returns"]]

X_test_msci = test[["returns", "msci", "msci_lag1", "msci_lag2", "msci_lag3"]]

X_test_sent = test[["returns", "sentiment", "sentiment_lag1", "sentiment_lag2", "sentiment_lag3"]]

X_test_both = test[["returns", "sentiment", "sentiment_lag1", "sentiment_lag2", "sentiment_lag3", "msci", "msci_lag1", "msci_lag2", "msci_lag3"]]

In [88]:
X_train_base = X_train_base.reshape((X_train_base.shape[0], 1, X_train_base.shape[1]))
X_test_base = X_test_base.reshape((X_test_base.shape[0], 1, X_test_base.shape[1]))

X_train_msci = X_train_msci.reshape((X_train_msci.shape[0], 1, X_train_msci.shape[1]))
X_test_msci = X_test_msci.reshape((X_test_msci.shape[0], 1, X_test_msci.shape[1]))

X_train_sent = X_train_msci.values.reshape((X_train_sent.shape[0], 1, X_train_sent.shape[1]))
X_test_sent = X_test_msci.values.reshape((X_test_sent.shape[0], 1, X_test_sent.shape[1]))

X_train_both = X_train_msci.values.reshape((X_train_both.shape[0], 1, X_train_sent.shape[1]))
X_test_both = X_test_msci.values.reshape((X_test_beoth.shape[0], 1, X_test_both.shape[1]))

ValueError: cannot reshape array of size 470 into shape (94,1,1)

In [54]:
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(X_train, y_train, epochs=100, batch_size=72, validation_data=(X_test, y_test), verbose=0, shuffle=False)

In [ ]:
#base

model_base = Sequential()
model_base.add(LSTM(50, input_shape=(X_train_base.shape[1], X_train_base.shape[2])))
model_base.add(Dense(1))

# Compile the model
model_base.compile(loss='mean_squared_error', optimizer='adam')
history_base = model.fit(X_train_base, y_train_base, epochs=100, batch_size=72, validation_data=(X_test, y_test), verbose=0, shuffle=False)

#msci

model_msci = Sequential()
model_msci.add(LSTM(50, input_shape=(X_train_msci.shape[1], X_train_msci.shape[2])))
model_msci.add(Dense(1))

# Compile the model
model_msci.compile(loss='mean_squared_error', optimizer='adam')
history_msci = model.fit(X_train_msci, y_train_msci, epochs=100, batch_size=72, validation_data=(X_test, y_test), verbose=0, shuffle=False)

#sent

model_sent = Sequential()
model_sent.add(LSTM(50, input_shape=(X_train_sent.shape[1], X_train_sent.shape[2])))
model_sent.add(Dense(1))

# Compile the model
model_sent.compile(loss='mean_squared_error', optimizer='adam')
history_sent = model.fit(X_train_sent, y_train_sent, epochs=100, batch_size=72, validation_data=(X_test, y_test), verbose=0, shuffle=False)

#both

model_both = Sequential()
model_both.add(LSTM(50, input_shape=(X_train_both.shape[1], X_train_both.shape[2])))
model_both.add(Dense(1))

# Compile the model
model_both.compile(loss='mean_squared_error', optimizer='adam')
history_both = model.fit(X_train_both, y_train_both, epochs=100, batch_size=72, validation_data=(X_test, y_test), verbose=0, shuffle=False)

In [64]:
'''from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import numpy as np

# Define a function to create the LSTM model
def create_model(units, epochs):
    model = Sequential()
    model.add(LSTM(units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=72, verbose=0, shuffle=False)
    return model

# Create a wrapper function for the create_model function
#def model_wrapper(units=50, epochs=100):
#    return create_model(units, epochs)

# Create a KerasRegressor based on the model_wrapper function
model = KerasRegressor(build_fn=create_model)


batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)


# Print the best parameters and best score
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)'''

'from keras.models import Sequential\nfrom keras.layers import LSTM, Dense\nfrom sklearn.model_selection import GridSearchCV\nfrom keras.wrappers.scikit_learn import KerasRegressor\nimport numpy as np\n\n# Define a function to create the LSTM model\ndef create_model(units, epochs):\n    model = Sequential()\n    model.add(LSTM(units, input_shape=(X_train.shape[1], X_train.shape[2])))\n    model.add(Dense(1))\n    model.compile(loss=\'mean_squared_error\', optimizer=\'adam\')\n    history = model.fit(X_train, y_train, epochs=epochs, batch_size=72, verbose=0, shuffle=False)\n    return model\n\n# Create a wrapper function for the create_model function\n#def model_wrapper(units=50, epochs=100):\n#    return create_model(units, epochs)\n\n# Create a KerasRegressor based on the model_wrapper function\nmodel = KerasRegressor(build_fn=create_model)\n\n\nbatch_size = [10, 20, 40, 60, 80, 100]\nepochs = [10, 50, 100]\nparam_grid = dict(batch_size=batch_size, epochs=epochs)\ngrid = GridSearchCV(

In [25]:
# Make predictions

#base

predicted_base = model_base.predict_base(X_test)
predicted_base

#msci

predicted_msci = model_msci.predict(X_test)
predicteds_sent

#sent

predicted_sent = model_sent.predict(X_test)
predicted_sent

#both

predicted_both = model_both.predict(X_test)
predicted_both

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


array([[0.32095987],
       [0.35959095],
       [0.5821033 ],
       [0.49729666],
       [0.59385574],
       [0.6139628 ],
       [0.61701375],
       [0.3420766 ],
       [0.40073952],
       [0.5773857 ],
       [0.564272  ],
       [0.4493974 ],
       [0.35351568],
       [0.47195736],
       [0.49739897],
       [0.379062  ],
       [0.39599437],
       [0.66906506],
       [0.49536777],
       [0.32168743],
       [0.50420105],
       [0.5153538 ],
       [0.57215846],
       [0.21488978],
       [0.5514621 ],
       [0.48305202],
       [0.3559257 ],
       [0.36740246],
       [0.3319923 ],
       [0.6374289 ],
       [0.3957451 ],
       [0.30530655],
       [0.5531315 ],
       [0.46002355],
       [0.2672071 ]], dtype=float32)

In [21]:
y_test

Date
2020-02-01    0.097432
2020-03-01    0.130260
2020-04-01    0.758229
2020-05-01    0.583780
2020-06-01    0.715974
2020-07-01    0.798056
2020-08-01    0.849651
2020-09-01    0.175000
2020-10-01    0.272177
2020-11-01    0.697413
2020-12-01    0.737130
2021-01-01    0.409402
2021-02-01    0.227381
2021-03-01    0.423632
2021-04-01    0.577166
2021-05-01    0.301280
2021-06-01    0.163470
2021-07-01    1.000000
2021-08-01    0.498985
2021-09-01    0.218997
2021-10-01    0.526515
2021-11-01    0.634755
2021-12-01    0.606659
2022-01-01    0.003174
2022-02-01    0.576372
2022-03-01    0.541125
2022-04-01    0.121997
2022-05-01    0.265636
2022-06-01    0.154506
2022-07-01    0.839230
2022-08-01    0.316956
2022-09-01    0.042934
2022-10-01    0.676721
2022-11-01    0.355164
2022-12-01    0.072915
Freq: MS, Name: returns, dtype: float64

In [22]:
y_test = np.squeeze(y_test)

predicted

array([[0.32095987],
       [0.35959095],
       [0.5821033 ],
       [0.49729666],
       [0.59385574],
       [0.6139628 ],
       [0.61701375],
       [0.3420766 ],
       [0.40073952],
       [0.5773857 ],
       [0.564272  ],
       [0.4493974 ],
       [0.35351568],
       [0.47195736],
       [0.49739897],
       [0.379062  ],
       [0.39599437],
       [0.66906506],
       [0.49536777],
       [0.32168743],
       [0.50420105],
       [0.5153538 ],
       [0.57215846],
       [0.21488978],
       [0.5514621 ],
       [0.48305202],
       [0.3559257 ],
       [0.36740246],
       [0.3319923 ],
       [0.6374289 ],
       [0.3957451 ],
       [0.30530655],
       [0.5531315 ],
       [0.46002355],
       [0.2672071 ]], dtype=float32)

In [23]:
y_test

Date
2020-02-01    0.097432
2020-03-01    0.130260
2020-04-01    0.758229
2020-05-01    0.583780
2020-06-01    0.715974
2020-07-01    0.798056
2020-08-01    0.849651
2020-09-01    0.175000
2020-10-01    0.272177
2020-11-01    0.697413
2020-12-01    0.737130
2021-01-01    0.409402
2021-02-01    0.227381
2021-03-01    0.423632
2021-04-01    0.577166
2021-05-01    0.301280
2021-06-01    0.163470
2021-07-01    1.000000
2021-08-01    0.498985
2021-09-01    0.218997
2021-10-01    0.526515
2021-11-01    0.634755
2021-12-01    0.606659
2022-01-01    0.003174
2022-02-01    0.576372
2022-03-01    0.541125
2022-04-01    0.121997
2022-05-01    0.265636
2022-06-01    0.154506
2022-07-01    0.839230
2022-08-01    0.316956
2022-09-01    0.042934
2022-10-01    0.676721
2022-11-01    0.355164
2022-12-01    0.072915
Freq: MS, Name: returns, dtype: float64

In [35]:

# Invert scaling if necessary for interpretation of the results
#predicted = np.squeeze(predicted)
#predicted_inv = scaler.inverse_transform(predicted)


rmse_base = np.sqrt(mean_squared_error(y_test, predicted_base))
print('Test RMSE: %.3f' % rmse_base)

rmse_msci = np.sqrt(mean_squared_error(y_test, predicted_msci))
print('Test RMSE: %.3f' % rmse_msci)

rmse_sent = np.sqrt(mean_squared_error(y_test, predicted_sent))
print('Test RMSE: %.3f' % rmse_sent)

rmse_both = np.sqrt(mean_squared_error(y_test, predicted_both))
print('Test RMSE: %.3f' % rmse_both)

Test RMSE: 0.159
